# Imports

Libraries

In [25]:
# Basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PyTorch libraries
import torch

#Neural network libraries
import torch.nn as nn
import torch.nn.functional as F

# #Creating optimizer
import torch.optim as optim

# For terminating program (useful for stopping before training is complete and seeing entire result)
import signal
import sys, os

CSV File

In [26]:
script_dir = os.path.abspath('') # absolute file path in Jupyter
file_path = os.path.join(script_dir, "AI EarthHack Dataset.csv") # file path for the text file input

dataset = pd.read_csv(file_path, header=None)
